In [25]:
import requests
from bs4 import BeautifulSoup
import re
import os
import chardet
import tempfile
import time
import datetime

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [26]:
# Authenticate PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [27]:
# Function to download and upload files
def download_and_upload_files():
    url = 'https://www.iea.org/data-and-statistics/data-product/world-energy-statistics-and-balances'

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    destination_folder_id = '1CRunUl_SszdENKlJD-SLU_IIOupxdCS6'

    file_count = 1
    for link in soup.find_all('a', href=re.compile('(pdf|txt)')):
        file_url = link['href']
        file = requests.get(file_url)

        filename = f"scheduled_downloads_{str(file_count).zfill(2)}"
        file_count += 1

        # Create a temporary file to store the binary content
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(file.content)

        f = drive.CreateFile({'title': filename, 'parents': [{'id': destination_folder_id}]})

        # Set the content of the file using the temporary file
        f.SetContentFile(temp_file.name)

        # Set the MIME type of the file (PDF in this case)
        f['mimeType'] = 'application/pdf'

        f.Upload()  # Upload the file to Google Drive

    print("Files downloaded and uploaded to Google Drive successfully.")

# Schedule the job to run on the 15th day of every month at 02:00 AM
while True:
    now = datetime.datetime.now()
    if now.day == 15 and now.hour == 2 and now.minute == 0:
        download_and_upload_files()
        time.sleep(60)  # Sleep for a minute to avoid repeated executions in the same minute
    else:
        time.sleep(60)  # Check every minute

KeyboardInterrupt: ignored